# Chapter 21

## 21.1
->的工作原理是什么? 或者说, "Hello" -> 42 和 42 -> "Hello"怎么会和对偶扯上关系?

A: "hello" -> 43会转化成("hello",43)是由于隐式转化的原因。`ArrowAssoc[A]`内定义了名为 `-> [B](y: B): Tuple2[A, B]`的方法，当使用到类似于 "hello" -> 43 的实例时函数被调用, 返回Tuple对象

## 21.2
定义一个操作符+%, 将一个给定的百分比添加到某个值. 举例来说,120 +% 10 等于132. 提示: 由于操作符是方法, 而不是函数, 你需要提供一个implicit

In [1]:
case class MyInt(v: Int) {
    def +%(percent: Int) = v + v / 100.0 * percent
}
  
implicit def int2MyInt(v: Int) = new MyInt(v)

120 +% 10

defined class MyInt
defined function int2MyInt
res0_2: Double = 132.0

## 21.3
定义一个!操作符, 计算某个整数的阶乘. 举例来说, 5!应得到120. 你将会需要一个经过丰富的类和一个隐式转换.

In [3]:
case class MyInt(v: Int) {
    def !(): Int = {
        if (v == 1) 1
        else v * (MyInt(v-1)!)
    }
}
  
implicit def int2MyInt(v: Int) = new MyInt(v)

5!

defined class MyInt
defined function int2MyInt
res2_2: Int = 120

## 21.4
有些人很喜欢那些读起来隐约像英语句子的流利API. 创建一个这样的API, 用来从控制台读取整数、浮点数以及字符串. 例如:
```
Read in aString askingFor "Your Name" and anInt askingFor "Your age" and aDouble askingFor "Your Weight"
```

In [3]:
class ReadOP {}
case object aString extends ReadOP
case object anInt extends ReadOP
case object aDouble extends ReadOP

class Read {
    private var typeOP: ReadOP = _
    var name = ""
    var age = 0
    var weight = 0.0
    
    def in(op: ReadOP): this.type = {
        typeOP = op
        this
    }
    
    def and(op: ReadOP): this.type = {
        typeOP = op
        this
    }
    
    def askingFor(s: String): this.type = {
        typeOP match {
            case _: aString.type => name = s
            case _: anInt.type => age = s.toInt
            case _: aDouble.type => weight = s.toDouble
        }
        this
    }
}

object Read { 
    def apply() = new Read() 
}

val r = Read()
r in aString askingFor "shizy" and anInt askingFor "35" and aDouble askingFor "144.4"
r.name
r.age
r.weight

defined class ReadOP
defined object aString
defined object anInt
defined object aDouble
defined class Read
defined object Read
r: wrapper.wrapper.Read = $sess.cmd2Wrapper$Helper$Read@62de9c0a
res2_7: wrapper.wrapper.Read = $sess.cmd2Wrapper$Helper$Read@62de9c0a
res2_8: String = "shizy"
res2_9: Int = 35
res2_10: Double = 144.4

## 21.5
提供执行下述运算所需要的代码:
```
smaller(Fraction(1,3), Fraction(3,1))
```
给出一个扩展自Ordered[Fraction]的RichFraction类

In [8]:
case class Fraction(val n: Int, val d: Int) {
    def *(that: Fraction) = new Fraction(this.n * that.n, this.d * that.d)
    override def toString() = "Fraction(" + n + ", " + d + ")"
}

def smaller[T](a: T, b: T)(implicit order: T => Ordered[T]) = if (order(a) < b) a else b

class RichFraction(val f: Fraction) extends Ordered[Fraction] {
    def compare(that: Fraction):Int = (f.n * that.d) - (that.n * f.d)
}

implicit def fraction2RichFraction(f: Fraction) = new RichFraction(f)

smaller(Fraction(3,1), Fraction(1,3))

defined class Fraction
defined function smaller
defined class RichFraction
defined function fraction2RichFraction
res7_4: wrapper.wrapper.Fraction = Fraction(1, 3)

## 21.6
比较java.awt.Point类的对象, 按词典顺序比较(即依次比较x坐标和y坐标的值)

In [19]:
import java.awt.Point
import scala.util.Sorting._

implicit object LexicalPointOrdering extends Ordering[Point] {
    def compare(a: Point, b: Point): Int = (a.x - b.x).signum match {
        case 0 => (a.y - b.y).signum
        case r => r
    }
}

val points = Array(new Point(4, 5), new Point(6, 5), new Point(1, 100), new Point(6, 4))

quickSort(points)(LexicalPointOrdering)

import java.awt.Point

import scala.util.Sorting._


defined object LexicalPointOrdering
points: Array[Point] = Array(
  java.awt.Point[x=1,y=100],
  java.awt.Point[x=4,y=5],
  java.awt.Point[x=6,y=4],
  java.awt.Point[x=6,y=5]
)

## 21.7
继续前一个练习, 根据两个点到原点的距离进行比较. 你如何在两种排序之间切换?

In [22]:
import java.awt.Point
import scala.util.Sorting._

class LexicalOrdered(val p: Point) extends Ordered[Point] {
    def compare(that: Point): Int = (p.x - that.x).signum match {
        case 0 => (p.y - that.y).signum
        case r => r
    }
}

class OriginOrdered(val p: Point) extends Ordered[Point] {
    private def distanceToOrigin(p: Point): Double = math.sqrt(math.pow(p.x, 2) + math.pow(p.y, 2))
    def compare(that: Point): Int = (distanceToOrigin(p) - distanceToOrigin(that)).signum
}

class OrderPointBy(val f: (Point) => Ordered[Point]) extends Ordering[Point] {
    implicit def point2Ordered(pt: Point): Ordered[Point] = f(pt)
    def compare(a: Point, b: Point): Int = a compare b
}

val points = Array(new Point(4, 5), new Point(6, 5), new Point(1, 100), new Point(6, 4))

quickSort(points)(new OrderPointBy((pt:Point) => new OriginOrdered(pt)))

import java.awt.Point

import scala.util.Sorting._


defined class LexicalOrdered
defined class OriginOrdered
defined class OrderPointBy
points: Array[Point] = Array(
  java.awt.Point[x=4,y=5],
  java.awt.Point[x=6,y=4],
  java.awt.Point[x=6,y=5],
  java.awt.Point[x=1,y=100]
)

## 21.8
在REPL中使用implicitly命令来召唤隐式对象. 你得到了哪些对象?

## 21.9
在Predef.scala中查找=:=对象,解释它的的工作原理

## 21.10
表达式`"abc".map(_.toUpper)`的结果是一个String, 但`"abc".map(_.toInt)`的结果是一个Vector. 为什么会这样?

In [23]:
"abc".map(_.toUpper)
"abc".map(_.toInt)

res22_0: String = "ABC"
res22_1: collection.immutable.IndexedSeq[Int] = Vector(97, 98, 99)

A: Builder[Char,B] += Char 后的结果 TraversableLike[Char, That] 会隐式转化成 String. 而Int 没有